# Softmax Regression

## Necessary Libraries

In [ ]:
%pip install scikit-learn

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


# Load the data and define variables


In [7]:
# Taining and testing data
train = pd.read_csv('data/sign_mnist_train.csv')
test = pd.read_csv('data/sign_mnist_test.csv')

# Size of the images
image_width = 28
image_height = 28

## Load training data

In [8]:
# training images
X = np.array(train.drop('label', axis=1))
# training labels
y = np.array(train['label'])

# number of examples
m = len(y)

# normalize the data
# Pixel values typically range from 0 to 255.
# Dividing by 255.0 scales the data to a [0, 1] range.
# Normalization improves the performance and convergence speed of many machine learning algorithms

pixel_max_value = 255.0

X = np.array(X , dtype='float')/ pixel_max_value

# number of different labels
n_classes = len(np.unique(y))

# we get 24 instead of 26 labels because we do not have "j" and "z" samples (maybe because they are movement signals).
# in order to not get empty labels lets rename the labels
for i in range(len(y)):
    if y[i] > 8:
        y[i] -= 1

print(f'Number of training examples: {m}')
print(f'Number of features: {X.shape[1]}')
print(f'Number of classes: {n_classes}')
print(f'X_shape: {X.shape}')
print(f'y_shape: {y.shape}')


Number of training examples: 27455
Number of features: 784
Number of classes: 24
X_shape: (27455, 784)
y_shape: (27455,)


## Load Test Data

In [9]:
X_test = np.array(test.drop('label', axis=1))
y_test = np.array(test['label'])

# number of test examples
m_test = len(y_test)

pixel_max_value = 255.0

X_test = np.array(X_test , dtype='float')/ pixel_max_value

n_test_classes = len(np.unique(y_test))

for i in range(len(y_test)):
    if y_test[i] > 8:
        y_test[i] -= 1


print(f'Number of test examples: {m_test}')
print(f'X_test_shape: {X_test.shape}')
print(f'y_test_shape: {y_test.shape}')

Number of test examples: 7172
X_test_shape: (7172, 784)
y_test_shape: (7172,)


## Model Train

In [10]:
model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)
model.fit(X, y)

# Predict on test data
y_pred = model.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Plot confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.title("Confusion Matrix - Softmax Regression")
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.show()

/home/rodrigoabreu/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Accuracy: 0.6940881204684886

Classification Report:
               precision    recall  f1-score   support

           0       0.88      1.00      0.94       331
           1       1.00      0.89      0.94       432
           2       0.87      0.91      0.89       310
           3       0.90      0.84      0.87       245
           4       0.88      0.87      0.88       498
           5       0.71      0.91      0.80       247
           6       0.82      0.79      0.80       348
           7       0.85      0.72      0.78       436
           8       0.58      0.61      0.59       288
           9       0.66      0.41      0.50       331
          10       0.65      0.90      0.75       209
          11       0.67      0.64      0.66       394
          12       0.66      0.57      0.61       291
          13       0.99      0.61      0.75       246
          14       0.90      0.95      0.92       347
          15       0.65      0.74      0.69       164
          16       0.17    

NameError: name 'sns' is not defined

<Figure size 1000x800 with 0 Axes>